In [1]:
import torch
from botorch.models import SingleTaskGP
from botorch.fit import fit_gpytorch_model
from gpytorch.mlls import ExactMarginalLogLikelihood
from lume_model.variables import ScalarVariable, DistributionVariable
from lume_model.models.gp_model import GPModel
from gpytorch.kernels import RBFKernel
from gpytorch.kernels import ScaleKernel

# Multi-output example

In [2]:
torch.manual_seed(0);

# Create training data, 1 input, 3 outputs
train_x = torch.rand(5, 1)  
train_y = torch.stack((
    torch.sin(train_x * (2 * torch.pi)) + 0.1 * torch.randn(1),  
    torch.cos(train_x * (2 * torch.pi)) + 0.1 * torch.randn(1),  
    torch.sin(2 * train_x * (2 * torch.pi)) + 0.1 * torch.randn(1)  
), dim=-1).squeeze(1) 


# Initialize the GP model
rbf_kernel = ScaleKernel(RBFKernel())

model = SingleTaskGP(
    train_x.to(dtype=torch.double), 
    train_y.to(dtype=torch.double),
    covar_module=rbf_kernel
)

# Fit the model
mll = ExactMarginalLogLikelihood(model.likelihood, model)
fit_gpytorch_model(mll)

# Derive posterior mean and variance
model.eval()
test_x = torch.linspace(0, 1, 10).reshape(-1, 1).to(dtype=torch.double)
posterior = model.posterior(test_x)

# Derive the posterior mean and variance for each output
mean = posterior.mean
variance = posterior.variance
print("Posterior Mean for each output:")
print(mean)
print("\nPosterior Variance for each output:")
print(variance)

/opt/miniconda3/envs/lume-latest-310/lib/python3.10/site-packages/botorch/models/utils/assorted.py:202: InputDataWarning: Input data is not standardized (mean = tensor([0.3013, 0.0843, 0.2563], dtype=torch.float64), std = tensor([0.7718, 0.7580, 0.7250], dtype=torch.float64)). Please consider scaling the input to zero mean and unit variance.
  warnings.warn(msg, InputDataWarning)


Posterior Mean for each output:
tensor([[ 0.1191,  1.0676,  0.8695],
        [ 0.6962,  0.7921,  0.6797],
        [ 1.0393,  0.2077,  0.3564],
        [ 0.9186, -0.4773,  0.0466],
        [ 0.3888, -0.9233, -0.1288],
        [-0.2651, -0.8889, -0.1557],
        [-0.7515, -0.4197, -0.0993],
        [-0.9433,  0.1902, -0.0232],
        [-0.8866,  0.6319,  0.0497],
        [-0.7060,  0.7829,  0.1197]], dtype=torch.float64,
       grad_fn=<CloneBackward0>)

Posterior Variance for each output:
tensor([[8.9528e-03, 6.5541e-03, 1.9921e-01],
        [1.6997e-04, 1.0506e-04, 1.0898e-01],
        [5.7277e-04, 3.9206e-04, 1.0253e-01],
        [3.9346e-04, 2.5698e-04, 1.2137e-01],
        [5.1715e-04, 3.8939e-04, 1.3695e-01],
        [1.4667e-03, 1.1957e-03, 1.5283e-01],
        [5.0446e-03, 4.6348e-03, 1.6268e-01],
        [5.1627e-04, 3.7501e-04, 1.9050e-01],
        [5.2062e-02, 5.0726e-02, 2.8656e-01],
        [2.2894e-01, 2.2627e-01, 4.3334e-01]], dtype=torch.float64,
       grad_fn=<CloneBac

## LUME-Model import

In [3]:
# Define input variables
input_variables = [ScalarVariable(name="x")]

# Define output variables
# Currently the "distribution_type" field doesn't do anything
output_variables = [
    DistributionVariable(name="output1", distribution_type="MultiVariateNormal"),
    DistributionVariable(name="output2", distribution_type="MultiVariateNormal"),
    DistributionVariable(name="output3", distribution_type="MultiVariateNormal")
]

# Create lume_model instance
gp_lume_model = GPModel(model=model, input_variables=input_variables, output_variables=output_variables)

### Evaluate model and run methods

In [4]:
input_dict = {"x": test_x.squeeze(1).to(dtype=torch.double)}

In [5]:
input_dict

{'x': tensor([0.0000, 0.1111, 0.2222, 0.3333, 0.4444, 0.5556, 0.6667, 0.7778, 0.8889,
         1.0000], dtype=torch.float64)}

In [6]:
# Evaluate function returns a dictionary mapping each output to a torch.distributions.Distribution
output_dict = gp_lume_model.evaluate(input_dict)

In [7]:
output_dict

{'output1': MultivariateNormal(loc: torch.Size([10]), covariance_matrix: torch.Size([10, 10])),
 'output2': MultivariateNormal(loc: torch.Size([10]), covariance_matrix: torch.Size([10, 10])),
 'output3': MultivariateNormal(loc: torch.Size([10]), covariance_matrix: torch.Size([10, 10]))}

In [8]:
test_prob = output_dict["output1"].sample(torch.Size([2]))

In [9]:
print("Posterior mean:", output_dict["output1"].mean)
print("Posterior Variance ", output_dict["output1"].variance)
print("Log Likelihood", output_dict["output1"].log_prob(test_prob))
print("Rsample ", output_dict["output1"].rsample(torch.Size([3])))

Posterior mean: tensor([ 0.1191,  0.6962,  1.0393,  0.9186,  0.3888, -0.2651, -0.7515, -0.9433,
        -0.8866, -0.7060], dtype=torch.float64, grad_fn=<ExpandBackward0>)
Posterior Variance  tensor([8.9528e-03, 1.6997e-04, 5.7277e-04, 3.9346e-04, 5.1715e-04, 1.4667e-03,
        5.0446e-03, 5.1627e-04, 5.2062e-02, 2.2894e-01], dtype=torch.float64,
       grad_fn=<ExpandBackward0>)
Log Likelihood tensor([23.0400, 23.1697], dtype=torch.float64, grad_fn=<SubBackward0>)
Rsample  tensor([[ 0.0907,  0.7043,  1.0711,  0.9269,  0.3688, -0.2822, -0.7675, -0.9718,
         -0.9131, -0.7421],
        [ 0.0608,  0.6952,  1.0342,  0.9255,  0.4047, -0.3430, -0.8944, -0.9312,
         -0.6536, -0.5014],
        [-0.0622,  0.6853,  1.0661,  0.9004,  0.3716, -0.2221, -0.6966, -0.9473,
         -0.8827, -0.5358]], dtype=torch.float64, grad_fn=<AddBackward0>)


### Outputs with original model

In [10]:
print("Posterior mean:", posterior.mean[:,0])
print("Posterior Variance ", posterior.variance[:,0])

Posterior mean: tensor([ 0.1191,  0.6962,  1.0393,  0.9186,  0.3888, -0.2651, -0.7515, -0.9433,
        -0.8866, -0.7060], dtype=torch.float64, grad_fn=<SelectBackward0>)
Posterior Variance  tensor([8.9528e-03, 1.6997e-04, 5.7277e-04, 3.9346e-04, 5.1715e-04, 1.4667e-03,
        5.0446e-03, 5.1627e-04, 5.2062e-02, 2.2894e-01], dtype=torch.float64,
       grad_fn=<SelectBackward0>)


# A 3D Rosenbrock example for GPModel class

## Create and train a GP

In [11]:
# Define the 3D Rosenbrock function
def rosenbrock(X):
    x1, x2, x3 = X[..., 0], X[..., 1], X[..., 2]
    return (1 - x1)**2 + 100 * (x2 - x1**2)**2 + (1 - x2)**2 + 100 * (x3 - x2**2)**2

In [12]:
# Generate training data
train_x = torch.rand(20, 3) * 4 - 2  # 20 points in 3D space, scaled to [-2, 2]
train_y = rosenbrock(train_x).unsqueeze(-1)  # Compute the Rosenbrock function values

# Define the GP model
gp_model = SingleTaskGP(train_x.to(dtype=torch.double), train_y.to(dtype=torch.double))

# Fit the model
mll = ExactMarginalLogLikelihood(gp_model.likelihood, gp_model)
fit_gpytorch_model(mll)

# Evaluate the model on test data
test_x = torch.rand(10, 3) * 4 - 2  # 10 new points in 3D space
gp_model.eval()
posterior = gp_model.posterior(test_x)

# Get the mean and variance of the posterior
mean = posterior.mean
variance = posterior.variance

print("Posterior mean: ", mean)
print("Posterior variance: ", variance)

/opt/miniconda3/envs/lume-latest-310/lib/python3.10/site-packages/botorch/models/utils/assorted.py:174: InputDataWarning: Input data is not contained to the unit cube. Please consider min-max scaling the input data.
  warnings.warn(msg, InputDataWarning)
/opt/miniconda3/envs/lume-latest-310/lib/python3.10/site-packages/botorch/models/utils/assorted.py:202: InputDataWarning: Input data is not standardized (mean = tensor([667.0384], dtype=torch.float64), std = tensor([638.0577], dtype=torch.float64)). Please consider scaling the input to zero mean and unit variance.
  warnings.warn(msg, InputDataWarning)


Posterior mean:  tensor([[709.2621],
        [743.0249],
        [816.8719],
        [877.1305],
        [599.5735],
        [927.9536],
        [755.8055],
        [733.1906],
        [670.7312],
        [804.1502]], dtype=torch.float64, grad_fn=<UnsqueezeBackward0>)
Posterior variance:  tensor([[1061.7684],
        [1292.8560],
        [1239.0013],
        [1197.7443],
        [1042.3683],
        [1216.5796],
        [1220.7746],
        [1243.2245],
        [1091.1481],
        [1139.7435]], dtype=torch.float64, grad_fn=<UnsqueezeBackward0>)


## LUME-Model import

In [13]:
# Define input variables
input_variables = [ScalarVariable(name="x1"), ScalarVariable(name="x2"), ScalarVariable(name="x3")]

# Define output variables
# Currently the "distribution_type" field doesn't do anything
output_variables = [DistributionVariable(name="output1", distribution_type="MultiVariateNormal")]

# Create lume_model instance
gp_lume_model = GPModel(model=gp_model, input_variables=input_variables, output_variables=output_variables)

#### Evaluate model and run methods

In [14]:
input_x = torch.rand(3, 3) * 4 - 2  # 3 new points in 3D space
input_dict = {"x1": input_x[:,0].to(dtype=torch.double),
              "x2": input_x[:,1].to(dtype=torch.double),
              "x3": input_x[:,2].to(dtype=torch.double)
             }

In [15]:
# Evaluate function returns a dictionary mapping each output to a torch.distributions.Distribution
lume_dist = gp_lume_model.evaluate(input_dict)["output1"]

In [16]:
rand_test = torch.rand(1, 3)

In [17]:
print("Posterior mean:", lume_dist.mean)
print("Posterior Variance ", lume_dist.variance)
print("Log Likelihood", lume_dist.log_prob(rand_test))
print("Rsample ", lume_dist.rsample(torch.Size([3])))

Posterior mean: tensor([641.3173, 759.6992, 858.2711], dtype=torch.float64,
       grad_fn=<ViewBackward0>)
Posterior Variance  tensor([1157.0986, 1205.8884, 1203.6263], dtype=torch.float64,
       grad_fn=<ExpandBackward0>)
Log Likelihood tensor([-632.0540], dtype=torch.float64, grad_fn=<SubBackward0>)
Rsample  tensor([[649.7874, 771.1869, 917.4273],
        [659.0826, 787.8484, 819.4870],
        [664.5120, 787.8166, 925.1078]], dtype=torch.float64,
       grad_fn=<ViewBackward0>)


### Outputs with original model

In [18]:
posterior = gp_model.posterior(input_x)
botorch_dist = posterior.distribution

In [19]:
print("Posterior mean:", botorch_dist.mean)
print("Posterior Variance ", botorch_dist.variance)
print("Log Likelihood", botorch_dist.log_prob(rand_test))
print("Rsample ", botorch_dist.rsample(torch.Size([3])))

Posterior mean: tensor([641.3173, 759.6992, 858.2711], dtype=torch.float64,
       grad_fn=<ViewBackward0>)
Posterior Variance  tensor([1157.0986, 1205.8884, 1203.6263], dtype=torch.float64,
       grad_fn=<ExpandBackward0>)
Log Likelihood tensor([-632.0540], dtype=torch.float64, grad_fn=<SubBackward0>)
Rsample  tensor([[634.2593, 713.9684, 836.3392],
        [639.7731, 767.2576, 822.1241],
        [610.8663, 755.6557, 825.8629]], dtype=torch.float64,
       grad_fn=<ViewBackward0>)
